In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/Experiments/'

In [32]:
%matplotlib inline
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras as keras
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score,confusion_matrix,accuracy_score
from tensorflow.keras.applications import VGG16, ResNet50,VGG19,ResNet50V2,EfficientNetB0
from sklearn.preprocessing import StandardScaler
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
from Visualize import *
from Preprocess import *
import soundfile as sf
from pathlib import *
import pandas as pd
import numpy as np
import librosa
import glob
import shutil
import os
import cv2

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = 12

In [3]:
INPUT_PATH = Path('/net/store/cbc/projects/Pan troglodytes/audio_PH_dataset/PH/good_data/')
file_path =  '/net/store/cv/users/ybrima/scratch/data/Luango_Speaker/clip_loango.npz'
file_path2 =  '/net/store/cv/users/ybrima/scratch/data/Luango_Speaker/clip_loango_resized.npz'
BASE_PATH =  '/net/store/cv/users/ybrima/scratch/data/Luango_Speaker/'
FRAME_LENGHT =  1024
SAMPLE_RATE =  44100
HOP_LENGTH =  512

In [4]:
figures = Path('./Figures/')

In [5]:
data =  np.load(file_path, allow_pickle=True)
X = data['x']
y = data['y']
Z =  data['z']
CLASSES = list(data['c'])

In [19]:
def make_model(input_shape, output_nums):
    K.clear_session() # Clear previous models from memory.
    base_model = VGG16(weights=None, include_top=False, input_shape=input_shape)
    x = base_model.output
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(output_nums, activation='softmax')(x)
    model = keras.Model(inputs=base_model.input, outputs=x)
    optim_params = dict(learning_rate = 0.003,momentum = 0.9394867962846013,decay = 0.0003)
    model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.SGD(**optim_params),metrics=[keras.metrics.SparseCategoricalAccuracy(name="accuracy")]) #keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='sparse_top_5_categorical_accuracy')
    return model

In [7]:
X =  (X - X.mean(axis=0, keepdims=True))/X.std(axis=0,keepdims=True)

In [8]:
input_shape =  (X.shape[1],X.shape[2],1)
output_nums =  len(CLASSES)
X = X[...,np.newaxis]

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.1, shuffle=True)

In [24]:
model =  make_model(input_shape, output_nums)

In [25]:
history =  model.fit(x=X_train,y=y_train, batch_size=8, epochs=20, validation_split=0.1,verbose=1 )
model.evaluate(x=X_test,y=y_test)

Epoch 1/20


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/block1_conv1/Conv2D
 (defined at /net/store/cv/users/ybrima/miniconda3/envs/CV/lib/python3.9/site-packages/keras/layers/convolutional.py:231)
]] [Op:__inference_train_function_4967]

Errors may have originated from an input operation.
Input Source operations connected to node model/block1_conv1/Conv2D:
In[0] IteratorGetNext (defined at /net/store/cv/users/ybrima/miniconda3/envs/CV/lib/python3.9/site-packages/keras/engine/training.py:866)	
In[1] model/block1_conv1/Conv2D/ReadVariableOp:

Operation defined at: (most recent call last)
>>>   File "/net/store/cv/users/ybrima/miniconda3/envs/CV/lib/python3.9/runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/net/store/cv/users/ybrima/miniconda3/envs/CV/lib/python3.9/runpy.py", line 87, in _run_code

In [13]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
device_name

'/device:GPU:0'

In [18]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(Z[0],n_fft=HOP_LENGTH//2, hop_length=HOP_LENGTH)), ref=np.max)
D.shape 

(129, 87)

In [31]:
D = librosa.stft(Z[0],hop_length=HOP_LENGTH,n_fft= FRAME_LENGHT//4)  # STFT of y
S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
S_db.shape

(129, 87)